In [1]:

from rasterstats import zonal_stats
import rasterio
import geopandas as gpd
import numpy as np

# Count frequency
# Define the file paths for the shapefile
kommun_file = 'Kommun_Sweref99TM_region.shp'
# Read the shapefiles using geopandas
kommun_shapefile = gpd.read_file(kommun_file)
# Convert the coordinate reference system (CRS) to WGS84
kommun_shapefile = kommun_shapefile.to_crs("EPSG:4326")

def kommun_average(dataset, col):
    lat_grid = np.linspace(55.3376, 69.1763, 100) 
    lon_grid = np.linspace(11, 24.113, 100)

    # Compute the average cell size for latitude and longitude
    lat_cell_size = np.mean(np.diff(lat_grid))
    lon_cell_size = np.mean(np.diff(lon_grid))

    # Compute the affine transform with different cell sizes for latitude and longitude
    affine_transform = rasterio.Affine(lon_cell_size, 0, lon_grid[0], 0, -lat_cell_size, lat_grid[-1])

    nodata = np.nan
    
    all_time= pd.DataFrame(columns=[col,'time','municipality'])

    for t in range(len(dataset[col].values)):
        
        data=dataset[col].values[t]
        data[data==0]=np.nan
        data=np.flip(data, axis=0) # needed to correct projection
        time= pd.Timestamp(dataset[col].time.values[t]).strftime('%Y')
        print(time)

        # Create a dictionary with the data values and their corresponding spatial coordinates
        coords = {
            'x': np.tile(lon_grid, len(lat_grid)),
            'y': np.repeat(lat_grid, len(lon_grid)),
            'z': data.flatten()
        }

        kommun_stats = zonal_stats(kommun_shapefile, data, nodata=nodata, stats='mean', all_touched=True, affine=affine_transform)

        # Add the mean values as a new column in the GeoDataFrames

        temp_kommun=pd.DataFrame(columns=[col,'time'])
        temp_kommun[col] = [s['mean'] for s in kommun_stats]
        temp_kommun['time'] = time
        temp_kommun['municipality'] = kommun_shapefile.KnNamn
        all_time = pd.concat([all_time, temp_kommun], ignore_index=True)
    return all_time

In [1]:
# Average per municipality every time step and save to excel table
# Read shapefiles
import geopandas as gpd
import numpy as np
import os
from rasterstats import zonal_stats
import rasterio
def kommun_average2(dataset, col):
    # Get the current file directory (where the notebook is running)
    current_dir = os.getcwd()
    # Define the paths dynamically
    shapefiles_path = os.path.join(current_dir , "shapefiles", "Kommun_Sweref99TM_region.shp")
    # Read the shapefiles using geopandas
    kommun_shapefile = gpd.read_file(shapefiles_path)
    # Convert the coordinate reference system (CRS) to WGS84
    kommun_shapefile = kommun_shapefile.to_crs("EPSG:4326")
    lat_grid = np.linspace(55.3376, 69.1763, 100) 
    lon_grid = np.linspace(11, 24.113, 100)
    # Compute the average cell size for latitude and longitude
    lat_cell_size = np.mean(np.diff(lat_grid))
    lon_cell_size = np.mean(np.diff(lon_grid))
    # Compute the affine transform with different cell sizes for latitude and longitude
    affine_transform = rasterio.Affine(lon_cell_size, 0, lon_grid[0], 0, -lat_cell_size, lat_grid[-1])
    nodata = np.nan
    all_time= pd.DataFrame(columns=[col,'time','municipality'])
    for t in range(len(dataset.values)):
        
        data=dataset.values[t]
        data[data==0]=np.nan
        data=np.flip(data, axis=0) # needed to correct projection
        #time=dataset['time'].values[t]
        time= pd.Timestamp(dataset.time.values[t]).strftime('%Y')
        print(time)

        # Create a dictionary with the data values and their corresponding spatial coordinates
        coords = {
            'x': np.tile(lon_grid, len(lat_grid)),
            'y': np.repeat(lat_grid, len(lon_grid)),
            'z': data.flatten()
        }
        kommun_stats = zonal_stats(kommun_shapefile, data, nodata=nodata, stats='mean', all_touched=True, affine=affine_transform)
        temp_kommun=pd.DataFrame(columns=[col,'time'])
        temp_kommun[col] = [s['mean'] for s in kommun_stats]
        temp_kommun['time'] = time
        temp_kommun['municipality'] = kommun_shapefile.KnNamn
        all_time = pd.concat([all_time, temp_kommun], ignore_index=True)
    return all_time

In [2]:
# Please find the netCDF datasets at 10.6084/m9.figshare.28100255
# Use annual values instead (sum, max, average)
# Considering index at all times
import xarray as xr
import pandas as pd
import numpy as np
# Read Hazard Data
def read_data(month_number):
    # Heat waves
    hw_int= xr.open_dataset('tot_hw_intensity.nc')
    hw_int= hw_int.drop_duplicates(dim='time')
    hw_int = hw_int.sel(time=slice('2005', None))
    hw_int_yr = hw_int['heatwave index'].resample(time='M').max('time') 
    hw_int_yr = hw_int_yr.sel(time=hw_int_yr['time.month'] == month_number)

    # Droughts
    dr_int= xr.open_dataset('SPEI12_2024.nc')
    dr_int = dr_int.sel(time=slice('2005', None))
    dr_int_yr12 = dr_int.sel(time=dr_int['time.month'] == month_number)
    
    # Floods
    fl_int= xr.open_dataset('DFI_2024.nc')
    fl_int['time'] = pd.to_datetime(fl_int['time'])
    fl_int =fl_int.sortby('time')
    fl_int= fl_int.drop_duplicates(dim='time')
    fl_int = fl_int.sel(time=slice('2005', None))
    fl_int_yr = fl_int['flood_index'].resample(time='M').max('time')
    fl_int_yr = fl_int_yr.sel(time=fl_int_yr['time.month'] == month_number)
    
    return hw_int_yr, fl_int_yr, dr_int_yr12



In [4]:
hw_int

<xarray.Dataset>
Dimensions:         (time: 36525, lat: 100, lon: 100)
Coordinates:
  * time            (time) datetime64[ns] 1922-01-01 1922-01-02 ... 2021-12-31
  * lat             (lat) float64 55.34 55.48 55.62 55.76 ... 68.9 69.04 69.18
  * lon             (lon) float64 11.0 11.13 11.26 11.4 ... 23.85 23.98 24.11
Data variables:
    heatwave index  (time, lat, lon) float64 ...

In [11]:
months=[1,2,3,4,5,6,7,8,9,10,11,12]
for month_number in months:
    hw_int_yr, fl_int_yr, dr_int_yr12= read_data(month_number)
    print('Month: '+ str(month_number))
    all_kommun_dr12=kommun_average(dr_int_yr12, 'SPEI')
    all_kommun_hw=kommun_average2(hw_int_yr, 'heatwave index')
    all_kommun_fl=kommun_average2(fl_int_yr, 'flood_index')

    df1=pd.DataFrame(columns=['Time', 'Variable', 'Municipality', 'Value'])
    df1['Time']=all_kommun_hw['time']
    df1['Variable']='HWI'
    df1['Value']=all_kommun_hw['heatwave index']
    df1['Municipality']=all_kommun_hw['municipality']

    df2=pd.DataFrame(columns=['Time', 'Variable', 'Municipality', 'Value'])
    df2['Time']=all_kommun_dr3['time']
    df2['Variable']='SPEI3'
    df2['Value']=all_kommun_dr3['SPEI']
    df2['Municipality']=all_kommun_dr3['municipality']

    df3=pd.DataFrame(columns=['Time', 'Variable', 'Municipality', 'Value'])
    df3['Time']=all_kommun_fl['time']
    df3['Variable']='DFI'
    df3['Value']=all_kommun_fl['flood_index']
    df3['Municipality']=all_kommun_fl['municipality']

    df4=pd.DataFrame(columns=['Time', 'Variable', 'Municipality', 'Value'])
    df4['Time']=all_kommun_dr12['time']
    df4['Variable']='SPEI12'
    df4['Value']=all_kommun_dr12['SPEI']#*(-1)
    df4['Municipality']=all_kommun_dr12['municipality']

    df=df1.append(df2)
    df=df.append(df3)
    df=df.append(df4)
    df.to_csv('Merged_hazards_mean_noconv_'+str(month_number)+'.csv', index=False)

Month: 1
2005
2006
2007
2008
2009
2010
2011
2012
2013
2014
2015
2016
2017
2018
2019
2020
2021
2022
2023
2005
2006
2007
2008
2009
2010
2011
2012
2013
2014
2015
2016
2017
2018
2019
2020
2021
2022
2023
2005
2006
2007
2008
2009
2010
2011
2012
2013
2014
2015
2016
2017
2018
2019
2020
2021
2022
2023
2005
2006
2007
2008
2009
2010
2011
2012
2013
2014
2015
2016
2017
2018
2019
2020
2021
2022
2023


C:\Users\MarlonVieiraPassos\AppData\Local\Temp\ipykernel_20244\3988798772.py:34: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df=df1.append(df2)
C:\Users\MarlonVieiraPassos\AppData\Local\Temp\ipykernel_20244\3988798772.py:35: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df=df.append(df3)
C:\Users\MarlonVieiraPassos\AppData\Local\Temp\ipykernel_20244\3988798772.py:36: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df=df.append(df4)


Month: 2
2005
2006
2007
2008
2009
2010
2011
2012
2013
2014
2015
2016
2017
2018
2019
2020
2021
2022
2023
2005
2006
2007
2008
2009
2010
2011
2012
2013
2014
2015
2016
2017
2018
2019
2020
2021
2022
2023
2005
2006
2007
2008
2009
2010
2011
2012
2013
2014
2015
2016
2017
2018
2019
2020
2021
2022
2023
2005
2006
2007
2008
2009
2010
2011
2012
2013
2014
2015
2016
2017
2018
2019
2020
2021
2022
2023


C:\Users\MarlonVieiraPassos\AppData\Local\Temp\ipykernel_20244\3988798772.py:34: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df=df1.append(df2)
C:\Users\MarlonVieiraPassos\AppData\Local\Temp\ipykernel_20244\3988798772.py:35: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df=df.append(df3)
C:\Users\MarlonVieiraPassos\AppData\Local\Temp\ipykernel_20244\3988798772.py:36: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df=df.append(df4)


Month: 3
2005
2006
2007
2008
2009
2010
2011
2012
2013
2014
2015
2016
2017
2018
2019
2020
2021
2022
2023
2005
2006
2007
2008
2009
2010
2011
2012
2013
2014
2015
2016
2017
2018
2019
2020
2021
2022
2023
2005
2006
2007
2008
2009
2010
2011
2012
2013
2014
2015
2016
2017
2018
2019
2020
2021
2022
2023
2005
2006
2007
2008
2009
2010
2011
2012
2013
2014
2015
2016
2017
2018
2019
2020
2021
2022
2023


...

C:\Users\MarlonVieiraPassos\AppData\Local\Temp\ipykernel_20244\3988798772.py:34: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df=df1.append(df2)
C:\Users\MarlonVieiraPassos\AppData\Local\Temp\ipykernel_20244\3988798772.py:35: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df=df.append(df3)
C:\Users\MarlonVieiraPassos\AppData\Local\Temp\ipykernel_20244\3988798772.py:36: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df=df.append(df4)


Month: 11
2005
2006
2007
2008
2009
2010
2011
2012
2013
2014
2015
2016
2017
2018
2019
2020
2021
2022
2023
2005
2006
2007
2008
2009
2010
2011
2012
2013
2014
2015
2016
2017
2018
2019
2020
2021
2022
2023
2005
2006
2007
2008
2009
2010
2011
2012
2013
2014
2015
2016
2017
2018
2019
2020
2021
2022
2023
2005
2006
2007
2008
2009
2010
2011
2012
2013
2014
2015
2016
2017
2018
2019
2020
2021
2022
2023


C:\Users\MarlonVieiraPassos\AppData\Local\Temp\ipykernel_20244\3988798772.py:34: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df=df1.append(df2)
C:\Users\MarlonVieiraPassos\AppData\Local\Temp\ipykernel_20244\3988798772.py:35: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df=df.append(df3)
C:\Users\MarlonVieiraPassos\AppData\Local\Temp\ipykernel_20244\3988798772.py:36: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df=df.append(df4)


Month: 12
2005
2006
2007
2008
2009
2010
2011
2012
2013
2014
2015
2016
2017
2018
2019
2020
2021
2022
2023
2005
2006
2007
2008
2009
2010
2011
2012
2013
2014
2015
2016
2017
2018
2019
2020
2021
2022
2023
2005
2006
2007
2008
2009
2010
2011
2012
2013
2014
2015
2016
2017
2018
2019
2020
2021
2022
2023
2005
2006
2007
2008
2009
2010
2011
2012
2013
2014
2015
2016
2017
2018
2019
2020
2021
2022
2023


C:\Users\MarlonVieiraPassos\AppData\Local\Temp\ipykernel_20244\3988798772.py:34: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df=df1.append(df2)
C:\Users\MarlonVieiraPassos\AppData\Local\Temp\ipykernel_20244\3988798772.py:35: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df=df.append(df3)
C:\Users\MarlonVieiraPassos\AppData\Local\Temp\ipykernel_20244\3988798772.py:36: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df=df.append(df4)
